- References
    - ["19-04 BERT를 이용한 키워드 추출 : 키버트(KeyBERT)", 딥 러닝을 이용한 자연어 처리 입문 (2022)](https://wikidocs.net/159467)
    - [ukairia777 GitHub - tensorflow-nlp-tutorial/19. Topic Modeling (LDA, BERT-Based)
/19-5. keybert_kor.ipynb](https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/df3f84702cd4e00ec2319549a2f029c3b2d666f6/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/19-5.%20keybert_kor.ipynb)

In [1]:
HUGGINGFACE_MODEL_PATH = 'klue/bert-base'
GPU_NUM = 1

특정 문서의 주요 정보를 이해하고자 할 때 키워드 추출을 통해서 입력 텍스트와 가장 관련이 있는 단어와 구문을 추출할 수 있습니다. 'Rake'나 'YAKE!'와 같은 키워드를 추출하는 데 사용할 수 있는 패키지가 이미 존재합니다. 그러나 이러한 모델은 일반적으로 텍스트의 통계적 특성에 기반하여 작동하며 의미적인 유사성에 대해서는 고려하지 않습니다. 의미적 유사성을 고려하기 위해서 여기서는 SBERT 임베딩을 활용하여 사용하기 쉬운 키워드 추출 알고리즘인 KeyBERT를 사용합니다.

In [2]:
import os
import torch
import random

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU_NUM}"  # Set the GPU number to use

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


# 1. 기본 KeyBERT

In [3]:
import numpy as np
import itertools
import re
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer

In [4]:
def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

In [5]:
def split_tokenizer(string):
    return string.split(" ")

In [6]:
doc = """ZZ 항공 소속 B7x7 항공기가 김포공항을 이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어 재운항하였으나, 운항 중 다시 시정이 착륙최저치 미만상태로 악화되어 김포공항으로 회항한 후 결항 및 환불 조치하였음."""

In [7]:
processed_doc = process_doc(doc)

In [8]:
tokenized_doc = " ".join(processed_doc.split(' '))
tokenized_doc

'ZZ 항공 소속 B7x7 항공기가 김포공항을 이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어 재운항하였으나, 운항 중 다시 시정이 착륙최저치 미만상태로 악화되어 김포공항으로 회항한 후 결항 및 환불 조치하였음.'

여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다.  CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 단쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [9]:
n_gram_range = (3, 8)

count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=split_tokenizer, token_pattern=None).fit([tokenized_doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 189
trigram 다섯개만 출력 : ['B7x7 항공기가 김포공항을' 'B7x7 항공기가 김포공항을 이륙하여' 'B7x7 항공기가 김포공항을 이륙하여 제주공항에'
 'B7x7 항공기가 김포공항을 이륙하여 제주공항에 접근' 'B7x7 항공기가 김포공항을 이륙하여 제주공항에 접근 중']


다음으로 이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 할 차례입니다. 한국어를 포함하고 있는 다국어 SBERT를 로드합니다.

In [10]:
HUGGINGFACE_MODEL_PATH

'klue/bert-base'

In [11]:
# 먼저 BERT 모델과 해당하는 토크나이저를 불러옵니다.
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertModel.from_pretrained(HUGGINGFACE_MODEL_PATH).to(device)

In [13]:
# 문장을 토큰화하고, BERT가 요구하는 형식에 맞도록 텐서로 변환합니다.
encoded_input = tokenizer(doc, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

# BERT 모델에 인코딩된 문장을 입력하고 결과를 얻습니다.
with torch.no_grad():
    model_output = model(**encoded_input)

# 모델 출력에서 문장 벡터를 얻습니다. 여기서는 [CLS] 토큰의 출력을 문장 벡터로 사용합니다.
doc_embedding = model_output.last_hidden_state[:, 0, :].cpu().numpy()

In [15]:
# 문장을 토큰화하고, BERT가 요구하는 형식에 맞도록 텐서로 변환합니다.
encoded_input = tokenizer(candidates.tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)

# BERT 모델에 인코딩된 문장을 입력하고 결과를 얻습니다.
with torch.no_grad():
    model_output = model(**encoded_input)

# 모델 출력에서 문장 벡터를 얻습니다. 여기서는 [CLS] 토큰의 출력을 문장 벡터로 사용합니다.
candidate_embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()

이제 문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다.

In [16]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [17]:
keywords

['제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.']

5개의 키워드가 출력되는데, 이들의 의미가 좀 비슷해보입니다. 비슷한 의미의 키워드들이 리턴되는 데는 이유가 있습니다. 당연히 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다. 만약, 지금 출력한 것보다는 좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 어쩌면 해당 키워드들이 문서를 잘 나타낼 가능성이 적을 수도 있습니다. 따라서 키워드들을 다양하게 출력하고 싶다면 키워드 선정의 정확성과 키워드들의 다양성 사이의 미묘한 균형이 필요합니다.

여기서는 다양한 키워드들을 얻기 위해서 두 가지 알고리즘을 사용합니다.

* Max Sum Similarity  
* Maximal Marginal Relevance

# 2. Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [18]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다.

낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [19]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['중 악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '악화되어 김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.']

그러나 상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [20]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['시정이 착륙최저치 미만상태로 악화되어 김포공항으로 회항한 후 결항',
 '인하여 김해공항으로 회항하였음. 회항 후 일시적으로 목적지 제주공항',
 '회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어',
 '제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.']

In [21]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['기상이 호전되어 재운항하였으나, 운항 중 다시 시정이 착륙최저치',
 '항공기가 김포공항을 이륙하여 제주공항에 접근 중 악시정으로 인하여',
 '회항한 후 결항 및 환불 조치하였음.',
 '인하여 김해공항으로 회항하였음. 회항 후 일시적으로 목적지 제주공항',
 '제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.']

# 3. Maximal Marginal Relevance

결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [22]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 우리가 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [23]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항']

In [24]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.3)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어',
 '악시정으로 인하여 김해공항으로 회항하였음. 회항',
 '접근 중 악시정으로 인하여 김해공항으로 회항하였음. 회항']

In [25]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.4)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어',
 '김포공항으로 회항한 후 결항 및 환불 조치하였음.',
 '시정이 착륙최저치 미만상태로 악화되어 김포공항으로 회항한 후 결항',
 '김해공항으로 회항하였음. 회항 후 일시적으로 목적지 제주공항']

In [26]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.5)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '제주공항 기상이 호전되어 재운항하였으나, 운항 중 다시 시정이',
 'ZZ 항공 소속',
 '후 결항 및 환불',
 '회항하였음. 회항 후 일시적으로 목적지 제주공항 기상이 호전되어']

In [27]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.6)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 'ZZ 항공 소속',
 '호전되어 재운항하였으나, 운항 중 다시 시정이',
 '후 결항 및 환불',
 '제주공항 기상이 호전되어']

그러나 상대적으로 높은 diversity값은 다양한 키워드 5개를 만들어냅니다.

In [28]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 'ZZ 항공 소속',
 '일시적으로 목적지 제주공항 기상이',
 '후 결항 및',
 '운항 중 다시 시정이 착륙최저치']

In [29]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.8)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 'ZZ 항공 소속',
 '중 다시 시정이',
 '후 결항 및',
 'ZZ 항공 소속 B7x7 항공기가 김포공항을']

In [30]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.9)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '중 다시 시정이',
 'ZZ 항공 소속',
 '후 결항 및',
 '항공 소속 B7x7 항공기가 김포공항을']

In [31]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=1.0)

['이륙하여 제주공항에 접근 중 악시정으로 인하여 김해공항으로 회항하였음.',
 '중 다시 시정이',
 'ZZ 항공 소속',
 '후 결항 및',
 '항공 소속 B7x7 항공기가 김포공항을']

# 4. 중간 정리
결론적으로, 테스트 케이스에 대해서는 **Max Sum Similarity** (`nr_candidates=30`)의 성능이 가장 우수한 것을 확인하였습니다.

위 과정을 정리하여 **document를 입력으로 받고, keyphrase를 반환하는 함수**를 만들어 봅시다.

In [32]:
GPU_NUM=1

In [33]:
import os
import torch
import random

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU_NUM}"  # Set the GPU number to use

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [39]:
import numpy as np
import itertools
import re
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

HUGGINGFACE_MODEL_PATH = 'oneonlee/KoAirBERT'

tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertModel.from_pretrained(HUGGINGFACE_MODEL_PATH).to(device)


def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

def split_tokenizer(string):
    return string.split(" ")

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [40]:
def doc_to_keyphrases(doc, model, n_gram_range=(1,10), keyword_top_n=5, mss_nr_candidates=30):
    processed_doc = process_doc(doc)
    tokenized_doc = " ".join(processed_doc.split(' '))
    
    count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=split_tokenizer, token_pattern=None).fit([tokenized_doc])

    candidates = count.get_feature_names_out()

    # 문장을 토큰화하고, BERT가 요구하는 형식에 맞도록 텐서로 변환합니다.
    encoded_input = tokenizer(doc, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
                                                                                                          
    # BERT 모델에 인코딩된 문장을 입력하고 결과를 얻습니다.
    with torch.no_grad():
        model_output = model(**encoded_input)
    # 모델 출력에서 문장 벡터를 얻습니다. 여기서는 [CLS] 토큰의 출력을 문장 벡터로 사용합니다.
    doc_embedding = model_output.last_hidden_state[:, 0, :].cpu().numpy()

    # 문장을 토큰화하고, BERT가 요구하는 형식에 맞도록 텐서로 변환합니다.
    encoded_input = tokenizer(candidates.tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
    # BERT 모델에 인코딩된 문장을 입력하고 결과를 얻습니다.
    with torch.no_grad():
        model_output = model(**encoded_input)
    # 모델 출력에서 문장 벡터를 얻습니다. 여기서는 [CLS] 토큰의 출력을 문장 벡터로 사용합니다.
    candidate_embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()

    # keyphrases_list = mmr(doc_embedding, candidate_embeddings, candidates, top_n=keyword_top_n, diversity=mmr_diversity)    
    keyphrases_list = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=keyword_top_n, nr_candidates=mss_nr_candidates)    
    
    return keyphrases_list

In [41]:
doc = """
아내에게 집에서 저녁식사를 하겠다고 하고, 집을 나선 헬리콥터 조종사는 '모두를 행복하게' 해줄 생각으로 집으로 돌아가기로 마음먹었지만 그 결정은 그다지 현명하지 못한 결정이 되어버렸다. 나는 헬리콥터를 집 뒤에 있는 마을 공터에 착륙시켰다. 공터 면적은 충분히 넓었고, 착륙은 교과서대로 진행하였으며, 착륙장소 주변엔 아무도 없었다. 같은 장소는 아니지만 전에도 아무 문제없이 착륙해 본 경험이 있어서 걱정 없이 착륙했는데. 착륙하고 보니 큰 일이 벌어져 있었다. 
지나가던 구급차가 내 헬리콥터가 착륙하는 장면을 보고는 경찰서에 추락신고를 한 것이다. 수 분만에 경찰차, 소방차, 기자들이 모여들었다. 
단지 가족과 저녁식사를 하려했다는 이유는 관계당국에 설득력 있는 이유가 되지 못했고 여러 가지 주정부 법에 따라 벌금형이 내려졌다. 
착륙 전 충분한 '주의'를 기울였음에도 불구하고 FAR91.13 조차도 위반했다고 한다.
"""

In [42]:
doc_to_keyphrases(doc, model=model)

["법에 따라 벌금형이 내려졌다. 착륙 전 충분한 '주의'를 기울였음에도",
 "'주의'를 기울였음에도 불구하고 FAR91.13 조차도 위반했다고 한다.",
 "따라 벌금형이 내려졌다. 착륙 전 충분한 '주의'를 기울였음에도 불구하고 FAR91.13",
 '착륙하는 장면을 보고는 경찰서에 추락신고를 한 것이다. 수 분만에 경찰차,',
 '본 경험이 있어서 걱정 없이 착륙했는데. 착륙하고']

# 5. 테스트셋 추론

In [43]:
import pandas as pd
from tqdm import tqdm

In [44]:
df = pd.read_csv("../data/rawdata/GYRO_testset.csv")

In [45]:
predicted_keyphrases_list = []
for i in tqdm(range(len(df))):
    doc = df["본문"][i]
    predicted_keyphrase = doc_to_keyphrases(doc, model=model)
    predicted_keyphrases_list.append('"' + '", "'.join(predicted_keyphrase) + '"')

100% 219/219 [13:21<00:00,  3.66s/it]


In [46]:
df["KeyBERT 예측 키워드"] = predicted_keyphrases_list

In [47]:
df.to_csv("../data/prediction/KeyBERT-BERT.csv", index=False)

In [48]:
exit()